In [11]:
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
InputData = np.loadtxt("dataInput.csv",delimiter=",")
OutputData = np.loadtxt ("dataOutput.csv", delimiter=",")

In [13]:
InputDataTrain = InputData[0:900]
InputDataTest = InputData[900:1106]

OutputDataTrain = OutputData[0:900]
OutputDataTest = OutputData[900:1106]

In [14]:
train = pd.read_csv("dataInput.csv", names= ["Filter1","Filter2","Filter3","Filter4","Filter5","Filter6","Filter7","Filter8","Filter9","Filter10","Filter11","Filter12","Filter13","Filter14","Filter15","Filter16","Filter17","Filter18","Filter19","Filter20","Filter21","Filter22","Filter23","Filter24","Filter25","Filter26","Filter27","Filter28","Filter29","Filter30","Filter31"])
#train

In [15]:
def create_model() :
    return tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(31, 1)),
      tf.keras.layers.Dense(62, activation='relu'),
      tf.keras.layers.Dense(62, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(31)
    ])

In [16]:
model = create_model()
model

In [24]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=InputDataTrain, 
          y=OutputDataTrain, 
          epochs=5, 
          validation_data=(InputDataTest, OutputDataTest), 
          callbacks=[tensorboard_callback])

Epoch 1/5

 2/29 [=>............................] - ETA: 6s - loss: 62.7573 - accuracy: 0.2031WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0010s vs `on_train_batch_end` time: 0.4868s). Check your callbacks.

29/29 [==============================] - 1s 23ms/step - loss: 64.4940 - accuracy: 0.1744 - val_loss: 67.9370 - val_accuracy: 0.2184
Epoch 2/5

29/29 [==============================] - 0s 2ms/step - loss: 57.7084 - accuracy: 0.2256 - val_loss: 59.7815 - val_accuracy: 0.2427
Epoch 3/5

29/29 [==============================] - 0s 2ms/step - loss: 53.3928 - accuracy: 0.2700 - val_loss: 53.7410 - val_accuracy: 0.2476
Epoch 4/5

29/29 [==============================] - 0s 3ms/step - loss: 48.3815 - accuracy: 0.2711 - val_loss: 48.3034 - val_accuracy: 0.2767
Epoch 5/5

29/29 [==============================] - 0s 2ms/step - loss: 44.5975 - accuracy: 0.3111 - val_loss: 44.0192 - val_accuracy: 0.3058


In [23]:
%tensorboard --logdir logs/fit

Launching TensorBoard...
Reusing TensorBoard on port 6006 (pid 14980), started 0:18:32 ago. (Use '!kill 14980' to kill it.)
Please visit http://localhost:6006 in a web browser.


In [22]:
kill 14980

  File "C:/Users/Robert/AppData/Local/Temp/xpython_8972/1648331085.py", line 1
    kill 14980
         ^
SyntaxError: invalid syntax

